In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata

from torchvision import transforms, datasets

In [ ]:
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')

print(f'Using Pytorch version: {torch.__version__}, Device: {DEVICE}')

Using Pytorch version: 2.0.0+cu118, Device: cuda


In [ ]:
# BATCH_SIZE -> 1개의 Mini-Batch 단위에 대해서 구성된 데이터의 갯수
# EPOCHS -> 존재하고 있는 Mini-Batch를 전부 이용해서 학습한 횟수

# Iteration -> 1개의 Mini-Batch를 통해 학습한 횟수

# 예를 들어, 전체 데이터가 1만개, Batch Size가 1000개일 때,
# 학습은 1 Epoch당 Iteration이 10회 발생함

BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
train_dataset = datasets.CIFAR10(
    root = "../data/CIFAR_10", # 저장할 디렉토리 지정
    train = True, # Train 데이터셋인지 Test 데이터셋인지 지정
    download = True, # 해당 데이터를 인터넷에서 다운로드해 이용할 것인지 지정
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(), # 이미지를 50% 확률로 좌우반전
        transforms.ToTensor(),  # 0~255값으로 이뤄진 픽셀값을 0~1로 변환하는 작업
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Tensor로 변화한 이미지에 정규화를 진행
    ])
)

test_dataset = datasets.CIFAR10(
    root = "../data/CIFAR_10",
    train = False,
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(), # 이미지를 50% 확률로 좌우반전
        transforms.ToTensor(),  # 0~255값으로 이뤄진 픽셀값을 0~1로 변환하는 작업
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Tensor로 변화한 이미지에 정규화를 진행
    ])
)

# 데이터를 iterator 객체로 변환시켜주는 함수
train_loader = torchdata.DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_loader = torchdata.DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False
)



Files already downloaded and verified


In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(8 * 8 * 16, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 10)

  def forward(self, x):
    # 2차원 배열을 Flatten
    x = x.view(-1, 8 * 8 * 16)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    # 분류 확률 계산함수
    x = F.log_softmax(x, dim = 1)
    return x

In [ ]:
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5) # output, lr(learning rate): 학습률
criterion = nn.CrossEntropyLoss() # 모델의 예측값과 분류된 실제값의 차이 계산

print(model)

Net(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)


In [ ]:
def train(model, train_loader, optimizer, log_interval):
  model.train() # 정의한 mlp 모델을 학습상태로 지정
  for batch_idx, (image, label) in enumerate(train_loader): # enumerate(리스트 인덱스 반환)
    # 이미지/레이블을 장비에 할당
    image = image.to(DEVICE)
    label = label.to(DEVICE)
    # optimizer 비우기
    optimizer.zero_grad()
    # 장비에 할당한 이미지 데이터를 MLP Input에 지정
    output = model(image)
    # 계산된 예측값과 레이블 데이터로 Loss 값 계산
    loss = criterion(output, label)
    loss.backward() # Back Propagation으로 계산된 Gradient 값 할당
    optimizer.step()

    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{}({:.0f}%)]\tTrain Loss: {:.6f}'.format(
          Epoch, 
          batch_idx * len(image), 
          len(train_loader.dataset), 
          100. * batch_idx / len(train_loader),
          loss.item()))
    

In [ ]:
def evaluate(model, test_loader):
  model.eval() # 정의한 mlp 모델을 평가상태로 지정
  test_loss = 0 # test_loader 내의 데이터로 Loss 값을 계산하기 위해서 test_loss를 0으로 지정
  correct = 0 # 올바른 class로 분류된 경우를 계산하기 위해 correct를 0으로 임시 지정

  with torch.no_grad(): # 평가상태에서 계산된 Gradient 값이 저장되지 않도록 설정
    for image, label in test_loader:
       # 이미지/레이블을 장비에 할당
      image = image.to(DEVICE)
      label = label.to(DEVICE)
      # 장비에 할당한 이미지 데이터를 MLP Input에 지정
      output = model(image)
      # 계산된 예측값과 레이블 데이터로 Loss 값 추가
      test_loss += criterion(output, label).item()
      # output 벡터 내에서 가장 큰 값이 같다면 예측 성공으로 판단
      prediction = output.max(1, keepdim = True)[1]
      # 예측 성공 시 correct값을 더하기
      correct += prediction.eq(label.view_as(prediction)).sum().item()

  # test_loss 값을 test_loader 내에 있는 Mini-Batch 개수만큼 나눠 평균 Loss값으로 계산
  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset) # test_loader 데이터 중 얼마나 맞춘 지 정확도 계산
  return test_loss, test_accuracy

In [ ]:
for Epoch in range(1, EPOCHS + 1):
  train(model, train_loader, optimizer, log_interval = 200)
  test_loss, test_accuracy = evaluate(model, test_loader)
  print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(Epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000(0%)]	Train Loss: 2.306349
Train Epoch: 1 [6400/50000(13%)]	Train Loss: 2.295599
Train Epoch: 1 [12800/50000(26%)]	Train Loss: 2.287020
Train Epoch: 1 [19200/50000(38%)]	Train Loss: 2.277914
Train Epoch: 1 [25600/50000(51%)]	Train Loss: 2.304658
Train Epoch: 1 [32000/50000(64%)]	Train Loss: 2.223799
Train Epoch: 1 [38400/50000(77%)]	Train Loss: 2.108488
Train Epoch: 1 [44800/50000(90%)]	Train Loss: 2.117427

[EPOCH: 1], 	Test Loss: 0.0640, 	Test Accuracy: 28.26 % 

Train Epoch: 2 [0/50000(0%)]	Train Loss: 2.102288
Train Epoch: 2 [6400/50000(13%)]	Train Loss: 2.067586
Train Epoch: 2 [12800/50000(26%)]	Train Loss: 1.913772
Train Epoch: 2 [19200/50000(38%)]	Train Loss: 1.978625
Train Epoch: 2 [25600/50000(51%)]	Train Loss: 1.744250
Train Epoch: 2 [32000/50000(64%)]	Train Loss: 1.728637
Train Epoch: 2 [38400/50000(77%)]	Train Loss: 1.681114
Train Epoch: 2 [44800/50000(90%)]	Train Loss: 1.414682

[EPOCH: 2], 	Test Loss: 0.0509, 	Test Accuracy: 41.22 % 

Train Epoch: 3